# Observações e configuração inicial

Material baseado no trabalho desenvolvido pelo Estadão - Como votou sua vizinhança https://github.com/estadao/como-votou-sua-vizinhanca



Este notebook só é necessário caso seja necessário alterar alguma configuração do arquivo com os locais de votação. Por exemplo, se for trabalhar com dados de outras eleições, que não a de 2018.

In [28]:
#Bibliotecas

# Acesso ao arquivos do Google Drive
# Na dúvida sobre o caminho para os arquivos, no menu da lateral esquerda, 
# clique no icone em formato de pasta, 
# navegue até a pasta ou arquivo desejado, ao clicar com o botação direito do mouse
# é possivel copiar o caminho
from google.colab import drive
drive.mount('/content/gdrive')

#Outras bibliotecas
import pandas as pd
import numpy as np

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2.1 Base por seção

In [29]:
dtype = {'SGL_UF':'str', 'COD_LOCALIDADE_IBGE':'str', 'LOCALIDADE_LOCAL_VOTACAO':'str', 'ZONA':'str','BAIRRO_ZONA_SEDE':'str', 
         'LATITUDE_ZONA':'float', 'LONGITUDE_ZONA':'float', 
         'NUM_LOCAL':'str', 'SITUACAO_LOCAL':'str', 'TIPO_LOCAL':'str', 'LOCAL_VOTACAO':'str', 
         'ENDERECO':'str', 'BAIRRO_LOCAL_VOT':'str', 'CEP':'str', 
         'LATITUDE_LOCAL':'float',  'LONGITUDE_LOCAL':'float', 
         'NUM_SECAO':'str', 'SECAO_AGREGADORA':'str', 'SECAO_AGREGADA':'str'
    }
base = pd.read_csv('/content/gdrive/MyDrive/dados/data/locais/local-votacao-08-08-2018.csv', encoding='Latin5', sep=';',decimal=',', dtype=dtype)

## 1.1.1 Correspondência TSE

In [30]:
#Acrescenta o Códito TSE de identificação dos municípios
correspondencia = pd.read_csv("/content/gdrive/MyDrive/dados/data/votos/correspondencia-tse-ibge.csv", dtype={'GEOCOD_IBGE':'str','COD_TSE':'str'})

correspondencia["COD_TSE"] = correspondencia.COD_TSE.str.zfill(5)

base = base.merge(correspondencia[['GEOCOD_IBGE', 'COD_TSE']], 
                      left_on='COD_LOCALIDADE_IBGE', right_on='GEOCOD_IBGE')

In [31]:
base.head(3)

,SGL_UF,COD_LOCALIDADE_IBGE,LOCALIDADE_LOCAL_VOTACAO,ZONA,BAIRRO_ZONA_SEDE,LATITUDE_ZONA,LONGITUDE_ZONA,NUM_LOCAL,SITUACAO_LOCAL,TIPO_LOCAL,...,ENDERECO,BAIRRO_LOCAL_VOT,CEP,LATITUDE_LOCAL,LONGITUDE_LOCAL,NUM_SECAO,SECAO_AGREGADORA,SECAO_AGREGADA,GEOCOD_IBGE,COD_TSE
0,AC,1200807,PORTO ACRE,1,PORTAL DA AMAZONIA,NaN,NaN,1015,Ativo,Convencional,...,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,576,NaN,NaN,1200807,01023
1,AC,1200807,PORTO ACRE,1,PORTAL DA AMAZONIA,NaN,NaN,1015,Ativo,Convencional,...,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,577,NaN,NaN,1200807,01023
2,AC,1200807,PORTO ACRE,1,PORTAL DA AMAZONIA,NaN,NaN,1015,Ativo,Convencional,...,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,578,NaN,NaN,1200807,01023


In [32]:
#Removendo conteúdo do endereço que atrapalha o georeferenciamento
base['ENDERECO'] = base.ENDERECO.str.replace(" - ZONA URBANA","")
base['ENDERECO'] = base.ENDERECO.str.replace(" - ZONA RURAL","")

In [33]:
# Cria um campo único para evitar consultas duplas
#base['local_unico'] = base.LOCAL_VOTACAO + ', ' + base.ENDERECO + ', ' + base.LOCALIDADE_LOCAL_VOTACAO + ', ' + base.SGL_UF
base['id_unico'] = base.COD_TSE + '-' + base.ZONA + '-' + base.NUM_LOCAL

In [8]:
# Cria um id único com base na query - permite juntar cada par de coordenadas a uma localidade única
#base['id_unico'] = base.groupby('local_unico').grouper.group_info[0].astype(str)

In [34]:
base = base[['SGL_UF', 'COD_LOCALIDADE_IBGE', 'COD_TSE', 'LOCALIDADE_LOCAL_VOTACAO', 
             'ZONA', 'NUM_SECAO', 'NUM_LOCAL' ,
              'LOCAL_VOTACAO',  'ENDERECO', 'BAIRRO_LOCAL_VOT', 'CEP', 
             'LATITUDE_LOCAL', 'LONGITUDE_LOCAL',
              'id_unico']]

In [35]:
base

,SGL_UF,COD_LOCALIDADE_IBGE,COD_TSE,LOCALIDADE_LOCAL_VOTACAO,ZONA,NUM_SECAO,NUM_LOCAL,LOCAL_VOTACAO,ENDERECO,BAIRRO_LOCAL_VOT,CEP,LATITUDE_LOCAL,LONGITUDE_LOCAL,id_unico
0,AC,1200807,01023,PORTO ACRE,1,576,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015
1,AC,1200807,01023,PORTO ACRE,1,577,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015
2,AC,1200807,01023,PORTO ACRE,1,578,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015
3,AC,1200807,01023,PORTO ACRE,1,579,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015
4,AC,1200807,01023,PORTO ACRE,1,580,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479511,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,25,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,NaN,NaN,73105-35-1015
479512,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,26,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,NaN,NaN,73105-35-1015
479513,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,27,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,NaN,NaN,73105-35-1015
479514,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,69,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,NaN,NaN,73105-35-1015


In [36]:
base.to_csv("/content/gdrive/MyDrive/dados/data/locais/local-votacao-08-08-2018-ajustado.csv", sep=';', index=False)

## 1.1.2 Segregerar os locais

In [41]:
# Cria uma cópia do campo, sem duplicatas - útil para geolocalizar cada endereço apenas uma vez
locais = base.drop_duplicates(subset='id_unico').reset_index()

#Cada local agrega várias seções, removendo a coluna das Seções para evitar qualquer confusão
locais.drop(columns='NUM_SECAO', axis=1, inplace=True)

In [42]:
locais

,index,SGL_UF,COD_LOCALIDADE_IBGE,COD_TSE,LOCALIDADE_LOCAL_VOTACAO,ZONA,NUM_LOCAL,LOCAL_VOTACAO,ENDERECO,BAIRRO_LOCAL_VOT,CEP,LATITUDE_LOCAL,LONGITUDE_LOCAL,id_unico
0,0,AC,1200807,01023,PORTO ACRE,1,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015
1,7,AC,1200807,01023,PORTO ACRE,1,1023,ESCOLA EDMUNDO PINTO DE ALMEIDA NETO,RODOVIA AC 10 KM 29 S/N - VILA DO INCRA,PROJETO HUMAITA,00000000,NaN,NaN,01023-1-1023
2,13,AC,1200807,01023,PORTO ACRE,1,1031,ESCOLA PUBLICA JADER SARAIVA MACHADO,PROJETO HUMAITA RAMAL DO V KM 37 S/N,PORTO ACRE,69921000,NaN,NaN,01023-1-1031
3,20,AC,1200807,01023,PORTO ACRE,1,1040,ESCOLA NOSSA SENHORA AUXILIADORA,ESTRADA DE PORTO ACRE KM 27 S/N - RAMAL DO FLA...,ZONA RURAL,69921000,NaN,NaN,01023-1-1040
4,22,AC,1200807,01023,PORTO ACRE,1,1058,SEDE DO INCRA,RODOVIA AC-10 KM-29,PROJETO HUMAITA,69921000,NaN,NaN,01023-1-1058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94703,479498,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1015,ESCOLA ESTADUAL PROFESSOR MANOEL SILVÉRIO DOURADO,"AV. HENRIQUE PEREIRA DA CRUZ, 560",CENTRO,77615000,NaN,NaN,92606-35-1015
94704,479505,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1023,ESCOLA MUNICIPAL HORÁCIO JOSÉ RODRIGUES,COMUNIDADE QUILOMBOLA BARRA DA AROEIRA,ZONA RURAL,77615000,NaN,NaN,92606-35-1023
94705,479507,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1031,ESCOLA MUNICIPAL FRANCISCO SOUSA BARROS,"PRAÇA DA BÍBLIA, S/N",CENTRO,77615000,NaN,NaN,92606-35-1031
94706,479510,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,NaN,NaN,73105-35-1015


In [43]:
# Transforma o campo da variável locais em string.
# Por algum motivo, ao fazer o subset, o dado voltou a ser tratado com int.
locais.id_unico = locais.id_unico.astype(str)

In [44]:
# Adiciona um campo para o país
locais['COUNTRY'] = 'BRASIL'

Preenche campos com valores temporários, mas que serão necessários para iniciar o processo de geolocalização.

In [45]:
def get_precision(row):

    # Função simples para determinar se a escola já tem um valor de lat/long
    # Se tiver, isso significa que ela já foi geocodificada pelo TSE
    # Assim, definimos uma categoria de precisão: "TSE"
    # As restantes serão classificadas, temporariamente, como None
    # E terão esse valor preenchido futuramente

    
    if np.isnan(row.LATITUDE_LOCAL):
        precision = 'NO_VALUE'
        fetched_address = 'NO_VALUE'
        lon = np.nan
        lat = np.nan
        
    else:
        precision = 'TSE'
        fetched_address = 'TSE'
        lon = row.LONGITUDE_LOCAL
        lat = row.LATITUDE_LOCAL
        
    return pd.Series({
            "lat":lat,
            "lon":lon,
            "fetched_address":fetched_address,
            "precision":precision,
            })

locais[["lat", "lon", "fetched_address", "precision"]] = locais.apply(get_precision, axis=1)

Deleta campos que se tornaram redundantes depois dessa preparação. Também reordena as colunas de modo mais intuitivo.

In [46]:
locais

,index,SGL_UF,COD_LOCALIDADE_IBGE,COD_TSE,LOCALIDADE_LOCAL_VOTACAO,ZONA,NUM_LOCAL,LOCAL_VOTACAO,ENDERECO,BAIRRO_LOCAL_VOT,CEP,LATITUDE_LOCAL,LONGITUDE_LOCAL,id_unico,COUNTRY,lat,lon,fetched_address,precision
0,0,AC,1200807,01023,PORTO ACRE,1,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,NaN,NaN,01023-1-1015,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
1,7,AC,1200807,01023,PORTO ACRE,1,1023,ESCOLA EDMUNDO PINTO DE ALMEIDA NETO,RODOVIA AC 10 KM 29 S/N - VILA DO INCRA,PROJETO HUMAITA,00000000,NaN,NaN,01023-1-1023,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
2,13,AC,1200807,01023,PORTO ACRE,1,1031,ESCOLA PUBLICA JADER SARAIVA MACHADO,PROJETO HUMAITA RAMAL DO V KM 37 S/N,PORTO ACRE,69921000,NaN,NaN,01023-1-1031,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
3,20,AC,1200807,01023,PORTO ACRE,1,1040,ESCOLA NOSSA SENHORA AUXILIADORA,ESTRADA DE PORTO ACRE KM 27 S/N - RAMAL DO FLA...,ZONA RURAL,69921000,NaN,NaN,01023-1-1040,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
4,22,AC,1200807,01023,PORTO ACRE,1,1058,SEDE DO INCRA,RODOVIA AC-10 KM-29,PROJETO HUMAITA,69921000,NaN,NaN,01023-1-1058,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94703,479498,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1015,ESCOLA ESTADUAL PROFESSOR MANOEL SILVÉRIO DOURADO,"AV. HENRIQUE PEREIRA DA CRUZ, 560",CENTRO,77615000,NaN,NaN,92606-35-1015,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
94704,479505,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1023,ESCOLA MUNICIPAL HORÁCIO JOSÉ RODRIGUES,COMUNIDADE QUILOMBOLA BARRA DA AROEIRA,ZONA RURAL,77615000,NaN,NaN,92606-35-1023,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
94705,479507,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1031,ESCOLA MUNICIPAL FRANCISCO SOUSA BARROS,"PRAÇA DA BÍBLIA, S/N",CENTRO,77615000,NaN,NaN,92606-35-1031,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE
94706,479510,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,NaN,NaN,73105-35-1015,BRASIL,NaN,NaN,NO_VALUE,NO_VALUE


In [48]:
locais = locais[['SGL_UF', 'COD_LOCALIDADE_IBGE', 'COD_TSE', 'LOCALIDADE_LOCAL_VOTACAO',
                 'ZONA', 'NUM_LOCAL', 'LOCAL_VOTACAO', 
                 'ENDERECO', 'BAIRRO_LOCAL_VOT', 'CEP', 'COUNTRY',
                 'id_unico', 
                 'lat', 'lon', 'fetched_address', 'precision'
                ]]

In [49]:
locais

,SGL_UF,COD_LOCALIDADE_IBGE,COD_TSE,LOCALIDADE_LOCAL_VOTACAO,ZONA,NUM_LOCAL,LOCAL_VOTACAO,ENDERECO,BAIRRO_LOCAL_VOT,CEP,COUNTRY,id_unico,lat,lon,fetched_address,precision
0,AC,1200807,01023,PORTO ACRE,1,1015,ESCOLA JOSE PLACIDO DE CASTRO,RUA DEPUTADO RAIMUNDO HERMINIO DE MELO S/N,PORTO ACRE,00000000,BRASIL,01023-1-1015,NaN,NaN,NO_VALUE,NO_VALUE
1,AC,1200807,01023,PORTO ACRE,1,1023,ESCOLA EDMUNDO PINTO DE ALMEIDA NETO,RODOVIA AC 10 KM 29 S/N - VILA DO INCRA,PROJETO HUMAITA,00000000,BRASIL,01023-1-1023,NaN,NaN,NO_VALUE,NO_VALUE
2,AC,1200807,01023,PORTO ACRE,1,1031,ESCOLA PUBLICA JADER SARAIVA MACHADO,PROJETO HUMAITA RAMAL DO V KM 37 S/N,PORTO ACRE,69921000,BRASIL,01023-1-1031,NaN,NaN,NO_VALUE,NO_VALUE
3,AC,1200807,01023,PORTO ACRE,1,1040,ESCOLA NOSSA SENHORA AUXILIADORA,ESTRADA DE PORTO ACRE KM 27 S/N - RAMAL DO FLA...,ZONA RURAL,69921000,BRASIL,01023-1-1040,NaN,NaN,NO_VALUE,NO_VALUE
4,AC,1200807,01023,PORTO ACRE,1,1058,SEDE DO INCRA,RODOVIA AC-10 KM-29,PROJETO HUMAITA,69921000,BRASIL,01023-1-1058,NaN,NaN,NO_VALUE,NO_VALUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94703,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1015,ESCOLA ESTADUAL PROFESSOR MANOEL SILVÉRIO DOURADO,"AV. HENRIQUE PEREIRA DA CRUZ, 560",CENTRO,77615000,BRASIL,92606-35-1015,NaN,NaN,NO_VALUE,NO_VALUE
94704,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1023,ESCOLA MUNICIPAL HORÁCIO JOSÉ RODRIGUES,COMUNIDADE QUILOMBOLA BARRA DA AROEIRA,ZONA RURAL,77615000,BRASIL,92606-35-1023,NaN,NaN,NO_VALUE,NO_VALUE
94705,TO,1719004,92606,SANTA TEREZA DO TOCANTINS,35,1031,ESCOLA MUNICIPAL FRANCISCO SOUSA BARROS,"PRAÇA DA BÍBLIA, S/N",CENTRO,77615000,BRASIL,92606-35-1031,NaN,NaN,NO_VALUE,NO_VALUE
94706,TO,1720150,73105,SÃO FÉLIX DO TOCANTINS,35,1015,ESCOLA ESTADUAL SAGRADO CORAÇÃO DE JESUS,"AV. PERNAMBUCO, QD 12, LT 05, S/N",CENTRO,77605000,BRASIL,73105-35-1015,NaN,NaN,NO_VALUE,NO_VALUE


In [50]:
locais.to_csv('/content/gdrive/MyDrive/dados/data/locais/local-votacao-08-08-2018-agregado.csv', index=False, sep=";")

In [ ]:
#Recarregar

In [4]:
base_locais = pd.read_csv('/content/gdrive/MyDrive/dados/data/locais/local-votacao-08-08-2018-agregado.csv', sep=';')

In [6]:
base_locais[base_locais['SGL_UF'] == 'SP']

,SGL_UF,COD_LOCALIDADE_IBGE,COD_TSE,LOCALIDADE_LOCAL_VOTACAO,ZONA,NUM_SECAO,NUM_LOCAL,LOCAL_VOTACAO,ENDERECO,BAIRRO_LOCAL_VOT,CEP,COUNTRY,local_unico,id_unico,lat,lon,fetched_address,precision
83303,SP,3550308,71072,SÃO PAULO,1,1,1015,COLÉGIO OBJETIVO,"AV. PAULISTA, 900",BELA VISTA,1310100,BRASIL,71072-1-1015,68015,NaN,NaN,NO_VALUE,NO_VALUE
83304,SP,3550308,71072,SÃO PAULO,1,28,1023,EE. RODRIGUES ALVES,"AV. PAULISTA, 227",BELA VISTA,1311000,BRASIL,71072-1-1023,68016,NaN,NaN,NO_VALUE,NO_VALUE
83305,SP,3550308,71072,SÃO PAULO,1,56,1040,EMEF. CELSO LEITE RIBEIRO FILHO,"RUA HUMAITÁ, 480",BELA VISTA,1321010,BRASIL,71072-1-1040,68017,NaN,NaN,NO_VALUE,NO_VALUE
83306,SP,3550308,71072,SÃO PAULO,1,71,1058,EE. MARIA JOSÉ,"RUA TREZE DE MAIO, 267",BELA VISTA,1327000,BRASIL,71072-1-1058,68018,NaN,NaN,NO_VALUE,NO_VALUE
83307,SP,3550308,71072,SÃO PAULO,1,87,1066,COLÉGIO JOÃO E RAPHAELA PASSALACQUA,"RUA JOÃO PASSALACQUA, 207",BELA VISTA,1326020,BRASIL,71072-1-1066,68019,NaN,NaN,NO_VALUE,NO_VALUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93833,SP,3545704,70157,SANTA ALBERTINA,427,8,1023,EE. CARLOS CELSO LENARDUZZI,"AV. FRANCISCO SCHIMIDT, 640",CENTRO,15750000,BRASIL,70157-427-1023,66963,NaN,NaN,NO_VALUE,NO_VALUE
93834,SP,3547650,62200,SANTA SALETE,427,47,1015,EE. FRANCISCO MOLINA MOLINA,"AV. PRESIDENTE ROOSEVELT, S/N",CENTRO,15768000,BRASIL,62200-427-1015,61582,NaN,NaN,NO_VALUE,NO_VALUE
93835,SP,3555802,72176,URÂNIA,427,15,1015,EE. PROF. AKIÓ SATORU,"RUA FORTALEZA, 549",CENTRO,15760000,BRASIL,72176-427-1015,70919,NaN,NaN,NO_VALUE,NO_VALUE
93836,SP,3555802,72176,URÂNIA,427,27,1023,EE. JOSÉ TEIXEIRA DO AMARAL,"RUA BOA ESPERANÇA, 371",CENTRO,15760000,BRASIL,72176-427-1023,70920,NaN,NaN,NO_VALUE,NO_VALUE
